In [ ]:
import requests
import pandas as pd
from datetime import datetime


In [17]:
import os
import requests
import pandas as pd
from io import StringIO
from sqlalchemy import create_engine
import pandas as pd
#from airflow.models import Variable

def fetch_grippe_data(type: str = "vaccin", save: bool = False):
    if type == "vaccin":
        url = "https://www.data.gouv.fr/fr/datasets/r/848e3e48-4971-4dc5-97c7-d856cdfde2f6"
    elif type == "cas":  
        url = "https://www.sentiweb.fr/api/v1/datasets/rest/incidence?indicator=3&geo=PAY&span=all"
    else:
        print("Type inconnu")
        return None

    headers = {"User-Agent": "Mozilla/5.0"}  # Évite certains blocages
    response = requests.get(url, headers=headers)

    if response.status_code == 200:
        try:
            data = StringIO(response.text)
            df = pd.read_csv(data, sep=",")  # Change `sep=";"` si nécessaire

            # Enregistrer le fichier si l'option est activée
            if save:
                save_path = "data"  # Dossier où enregistrer le fichier
                os.makedirs(save_path, exist_ok=True)  # Crée le dossier s'il n'existe pas
                filename = os.path.join(save_path, f"grippe_data_{type}.csv")  
                df.to_csv(filename, index=False, encoding="utf-8")
                print(f"✅ Fichier enregistré sous : {filename}")

            return df
        except Exception as e:
            print("❌ Erreur lors de la lecture du CSV :", e)
            return None
    else:
        print("❌ Erreur HTTP:", response.status_code)
        return None

# Exemple d'utilisation : récupérer et enregistrer le fichier dans `data/`
df = fetch_grippe_data("vaccin", save=True)

def enregistrer_dans_postgres():
    try:
        # Récupération des informations de connexion depuis Airflow Variables
       

        host="ep-dark-cell-a27yju5c.eu-central-1.pg.koyeb.app",  # Exemple: "postgres.koyeb.app"
        db="grippe_vaccins",  # Exemple: "spotify_data"
        login="koyeb-adm",     # Exemple: "postgres"
        port=5432
        password="npg_KfdxUp3W5bXI"     # Exemple: "password"

        # Création de la chaîne de connexion avec SQLAlchemy
        connection_string = f"postgresql://{login}:{password}@{host}:{port}/{db}"
        engine = create_engine(connection_string)

        # Définir le chemin du fichier CSV dans le dossier 'data'
        csv_path = os.path.join('data', 'grippe_data_vaccin.csv')  # ou le fichier spécifique que tu souhaites

        # Charger le fichier CSV dans un DataFrame
        df = pd.read_csv(csv_path)

        # Optionnel: Si tu veux vérifier les premières lignes du DataFrame
        print(df.head())  # Affiche les 5 premières lignes pour débogage

        # Enregistrer le DataFrame dans la table PostgreSQL
        df.to_sql(
            name='grippe_vaccins',   # Nom de la table dans la base PostgreSQL
            con=engine,                # Connexion à la base via SQLAlchemy
            if_exists='append',        # Ajouter les nouvelles lignes sans supprimer les anciennes
            index=False                # Ne pas ajouter l'index comme colonne
        )

        print("✅ Les données ont été enregistrées dans la base PostgreSQL avec succès.")

    except Exception as e:
        print(f"❌ Une erreur s'est produite : {e}")
enregistrer_dans_postgres()

✅ Fichier enregistré sous : data/grippe_data_vaccin.csv
❌ Une erreur s'est produite : No module named 'psycopg2'


In [3]:
 #host = Variable.get("koyeb_postgres_host")
       # login = Variable.get("koyeb_postgres_user")
      #  password = Variable.get("koyeb_postgres_password")
      #  port = Variable.get("koyeb_postgres_port")
     #   db = Variable.get("koyeb_postgres_db")
import os
import requests
import pandas as pd
from io import StringIO
from sqlalchemy import create_engine
import pandas as pd
def enregistrer_dans_postgres():
    try:
        # Récupération des informations de connexion depuis Airflow Variables

        host="ep-dark-cell-a27yju5c.eu-central-1.pg.koyeb.app"  # Exemple: "postgres.koyeb.app"
        db="grippe_vaccins"  # Exemple: "spotify_data"
        login="koyeb-adm"   # Exemple: "postgres"
        port=5432
        password="npg_KfdxUp3W5bXI"     # Exemple: "password"

        # Création de la chaîne de connexion avec SQLAlchemy
        connection_string = f"postgresql://{login}:{password}@{host}:{port}/{db}"
        engine = create_engine(connection_string)

        # Définir le chemin du fichier CSV dans le dossier 'data'
        csv_path = os.path.join('data', 'grippe_data_vaccin.csv')  # ou le fichier spécifique que tu souhaites

        # Charger le fichier CSV dans un DataFrame
        df = pd.read_csv(csv_path)

        # Optionnel: Si tu veux vérifier les premières lignes du DataFrame
        print(df.head())  # Affiche les 5 premières lignes pour débogage

        # Enregistrer le DataFrame dans la table PostgreSQL
        df.to_sql(
            name='grippe_vaccins',   # Nom de la table dans la base PostgreSQL
            con=engine,                # Connexion à la base via SQLAlchemy
            if_exists='append',        # Ajouter les nouvelles lignes sans supprimer les anciennes
            index=False                # Ne pas ajouter l'index comme colonne
        )

        print("✅ Les données ont été enregistrées dans la base PostgreSQL avec succès.")

    except Exception as e:
        print(f"❌ Une erreur s'est produite : {e}")
enregistrer_dans_postgres()

ModuleNotFoundError: No module named 'pandas'

In [21]:
!pip install psycopg2-binary

  Using cached psycopg2-binary-2.9.10.tar.gz (385 kB)
  Installing build dependencies ... done
  Getting requirements to build wheel ... error
  error: subprocess-exited-with-error
  
  × Getting requirements to build wheel did not run successfully.
  │ exit code: 1
  ╰─> [21 lines of output]
      running egg_info
      writing psycopg2_binary.egg-info/PKG-INFO
      writing dependency_links to psycopg2_binary.egg-info/dependency_links.txt
      writing top-level names to psycopg2_binary.egg-info/top_level.txt
      
      Error: pg_config executable not found.
      
      pg_config is required to build psycopg2 from source.  Please add the directory
      containing pg_config to the $PATH or specify the full executable path with the
      option:
      
          python setup.py build_ext --pg-config /path/to/pg_config build ...
      
      or with the pg_config option in 'setup.cfg'.
      
      If you prefer to avoid building psycopg2 from source, please install the PyPI
      '